# REFERENCES

* http://www.aic.uniovi.es/~jdiez/Jorge_Diez/Journal_Papers_files/luaces2012a.pdf

# AIM

文章データ（日本語）をBinary Relevanceの方法でマルチラベル分類を行う  
２値分類器にはナイーブベイズ分類を使う

In [3]:
import time
import glob
import numpy as np
import matplotlib
import matplotlib.pylab as plt
matplotlib.style.use("ggplot")
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.datasets import fetch_20newsgroups

In [52]:
from pkg_resources import get_distribution
import platform
print("python", platform.python_version())
print("")
libs = ["numpy", "matplotlib", "scikit-learn", "mecab-python3"]
for lib in libs:
    version = get_distribution(lib).version
    print(lib, version)

python 3.5.2

numpy 1.13.1
matplotlib 2.0.2
scikit-learn 0.19.0
mecab-python3 0.7


In [49]:
# 正例と負例のみを与えて学習させるようにする
class BinaryRelevance:
    def __init__(self, corpus):
        # クラスの初期化
        # :param corpus (string(object) np.array): コーパス
        self.labels = [] # 分類ラベルリスト
        self.clfs = {} # 分類器インスタンスリスト
        # MeCab
        self.index_category = 0
        self.index_root_form = 6
        self.target_categories = ["名詞", "動詞", "形容詞", "副詞", "連体詞", "感動詞"]
        self.stop_words = []
        self.mecab = MeCab.Tagger()
        # Vectorizer
        self.vectorizer = CountVectorizer(binary=True, analyzer=self.analyzer) # BoW, binary
        self.vectorizer.fit_transform(corpus)
        
    def analyzer(self, text):
        # 形態素解析を行う
        # :param text (string): 文章
        # :return:
        if not text:
            return []
        words = []
        node = self.mecab.parseToNode(text)
        while node:
            word = ""
            features = node.feature.split(",")
            if features[self.index_category] in self.target_categories:
                if features[self.index_root_form] == "*":
                    word = node.surface
                else:
                    word = features[self.index_root_form]
            if len(word) > 0 and word not in self.stop_words:
                words.append(word)
            node = node.next
        return words
        
    def train(self, target_label, positive_x, negative_x):
        # 学習
        # :param target_label (int): どのラベルの分類器を学習させるか
        # :param positive_x (string(object) np.array): 正例の文章リスト
        # :param negative_x (string(object) np.array): 負例の文章リスト
        # エラーチェック
        if not self.exists_label(target_label):
            return False
        # ペアデータセットにしてシャッフルする
        dataset = []
        for x in positive_x:
            dataset.append((x,1)) # 正例
        for x in negative_x:
            dataset.append((x,0)) # 負例
        dataset = np.array(dataset)
        np.random.shuffle(dataset) # シャッフル
        x = np.array(dataset[:,0], dtype="object") # 入力
        y = np.array(dataset[:,1], dtype="int32") # ラベル
        self.clfs[target_label].fit(self.vectorizer.transform(x), y) # 学習            
        return True
    
    def predict(self, x):
        # 予測
        # :param x: 予測させる文章リスト
        # :return: 
        result = []
        for i in range(len(x)):
            result.append([]) # 付与されたラベルを追加していくための配列
        for label in self.clfs: # 分類器をループ
            y = self.clfs[label].predict(self.vectorizer.transform(x)) # このラベルかどうかを予測
            for i, y_ in enumerate(y):
                if y_ == 1: # このラベルがつくと予想された
                    result[i].append(label)
        return result
        
    def set_labels(self, labels):
        # ラベルとラベルに対応する分類器インスタンスをセットする
        # :params labels (int np.array): 追加するラベルリスト
        for label in labels:
            self.labels.append(label) # ラベル追加
            self.clfs[label] = MultinomialNB(alpha=1.0) # 分類器インスタンス作成
    
    def exists_label(self, label):
        # ラベルが存在するかどうか
        # :param label (int): 調べるラベル
        if (label not in self.labels) or (label not in self.clfs):
            return False
        return True

In [65]:
path = "./livedoor_news_corpus/text"
categories = ["it-life-hack", "kaden-channel", "sports-watch"]
data = []
for category in categories:
    files = glob.glob(path + "/" + category + "/" + category + "*")
    for file in files:
        f = open(file)
        data.append((f.read(), categories.index(category))) # ファイルの中身、カテゴリーラベル
        f.close()
# ファイルの中身が、URL、日付、タイトル、本文と含んでいるので、本文のみにする
data_tmp = []
for d in data:
    x, t = d[0], d[1] # ファイルの中身、カテゴリーラベル
    x = x.split("\n")
    x = x[3:] # URL、日付、タイトルを落とす
    x = " ".join(x)
    data_tmp.append((x, t)) # 本文、カテゴリーラベル
data = data_tmp
data = np.array(data)
np.random.shuffle(data)
br = BinaryRelevance(data[:,0].tolist())
br.set_labels([0,1,2])

train0, train1, train2 = [], [], []
for d in data:
    x, t = d[0], int(d[1])
    if t == 0:
        train0.append(x)
    elif t == 1:
        train1.append(x)
    elif t == 2:
        train2.append(x)
train0, train1, tarin2 = train0[:N], train1[:N], train2[:N]

negative_x = np.array(train1+train2)
np.random.shuffle(negative_x)
negative_x = negative_x[:N]
br.train(0, train0, negative_x)

negative_x = np.array(train0+train2)
np.random.shuffle(negative_x)
negative_x = negative_x[:N]
br.train(1, train1, negative_x)

negative_x = np.array(train0+train1)
np.random.shuffle(negative_x)
negative_x = negative_x[:N]
br.train(2, train2, negative_x)

y = br.predict(data[:,0][-100:])
print(y)

[[2], [0, 1, 2], [2], [0, 1], [0, 1], [0, 1, 2], [2], [0, 1], [0, 1, 2], [0, 1], [2], [0, 1], [0, 1], [0, 1], [2], [0, 1], [0, 1], [0, 1, 2], [0, 2], [2], [0, 1, 2], [0, 1], [0, 1], [0, 1], [2], [0, 1], [0, 1, 2], [0, 1], [0, 1], [2], [0, 1], [2], [0, 1, 2], [0, 1], [2], [0, 1], [0, 1], [0, 1], [0, 1, 2], [2], [2], [2], [2], [0, 1], [0, 1], [2], [2], [0, 1], [0, 1, 2], [0, 1, 2], [0, 1], [0, 1], [2], [0, 1], [2], [0, 1], [0, 1], [0, 1], [2], [1, 2], [0, 1, 2], [0, 1], [0, 1], [0, 1], [2], [0, 1], [0, 2], [0, 1, 2], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [2], [2], [2], [0, 1, 2], [0, 1], [0, 1, 2], [0, 1, 2], [2], [2], [0, 1], [0, 1], [0, 1], [0, 1], [0, 2], [0, 1], [0, 1], [2], [0, 1], [0, 1], [0, 1], [0, 1, 2], [2], [0, 1], [2], [0, 1], [0, 1], [0, 1, 2]]
